In [1]:
import pyciemss

In [2]:
### SETUP ###

model_1_path = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/main/data/models/SEIRHD_NPI_Type1_petrinet.json"
start_time = 0.0
end_time = 100.
logging_step_size = 10.0
num_samples = 3

In [3]:
### SAMPLE INTERFACE ###

result = pyciemss.sample(model_1_path, end_time, logging_step_size, num_samples, start_time=start_time)
result["unprocessed_result"]

{'persistent_beta_c': tensor([0.7805, 0.3576, 0.6345]),
 'persistent_kappa': tensor([0.4398, 0.2051, 0.1710]),
 'persistent_gamma': tensor([0.4318, 0.4711, 0.3675]),
 'persistent_hosp': tensor([0.1515, 0.1108, 0.0642]),
 'persistent_death_hosp': tensor([0.0811, 0.0816, 0.0142]),
 'persistent_I0': tensor([6.6846, 9.1696, 6.1949]),
 'D_state': tensor([[0.4533, 1.3175, 2.2656, 3.2497, 4.2640, 5.3084, 6.3838, 7.4911, 8.6310],
         [0.2975, 0.5896, 0.7161, 0.7626, 0.7788, 0.7844, 0.7862, 0.7869, 0.7871],
         [0.0252, 0.0545, 0.0690, 0.0751, 0.0775, 0.0784, 0.0788, 0.0790, 0.0790]]),
 'E_state': tensor([[3.0450e+01, 3.1341e+01, 3.2269e+01, 3.3225e+01, 3.4208e+01, 3.5221e+01,
          3.6263e+01, 3.7336e+01, 3.6547e+01],
         [1.0990e+01, 3.7048e+00, 1.2507e+00, 4.2224e-01, 1.4254e-01, 4.8122e-02,
          1.6246e-02, 5.4845e-03, 1.6952e-03],
         [1.0999e+01, 4.2816e+00, 1.6762e+00, 6.5630e-01, 2.5696e-01, 1.0061e-01,
          3.9390e-02, 1.5422e-02, 5.7197e-03]]),
 'H_st

In [4]:
result['data'].head()

,timepoint_id,sample_id,persistent_beta_c_param,persistent_kappa_param,persistent_gamma_param,persistent_hosp_param,persistent_death_hosp_param,persistent_I0_param,D_state_state,E_state_state,H_state_state,I_state_state,R_state_state,S_state_state,infected_observable_state,exposed_observable_state,hospitalized_observable_state,dead_observable_state
0,0,0,0.780522,0.439797,0.431798,0.151517,0.08114,6.684645,0.453345,30.450066,4.715089,17.494068,62.825901,19339924.0,17.494068,30.450066,4.715089,0.453345
1,1,0,0.780522,0.439797,0.431798,0.151517,0.08114,6.684645,1.317504,31.340794,5.685230,18.023659,137.685303,19339846.0,18.023659,31.340794,5.685230,1.317504
2,2,0,0.780522,0.439797,0.431798,0.151517,0.08114,6.684645,2.265600,32.269176,5.966252,18.557556,215.428848,19339764.0,18.557556,32.269176,5.966252,2.265600
3,3,0,0.780522,0.439797,0.431798,0.151517,0.08114,6.684645,3.249676,33.224697,6.158191,19.107143,295.564789,19339682.0,19.107143,33.224697,6.158191,3.249676
4,4,0,0.780522,0.439797,0.431798,0.151517,0.08114,6.684645,4.263968,34.208332,6.342603,19.672852,378.085876,19339596.0,19.672852,34.208332,6.342603,4.263968


In [5]:
### SAMPLE INTERFACE with intervention ###

result = pyciemss.sample(model_1_path, end_time, logging_step_size, num_samples=int(5), start_time=start_time, static_state_interventions={1.: {"beta_c": 0.5}})
result["unprocessed_result"]

{'persistent_beta_c': tensor([0.6084, 0.4634, 0.5876, 0.4822, 0.5155]),
 'persistent_kappa': tensor([0.5758, 0.5119, 0.7357, 0.1904, 0.3618]),
 'persistent_gamma': tensor([0.1815, 0.4241, 0.4740, 0.1136, 0.2671]),
 'persistent_hosp': tensor([0.1320, 0.1961, 0.1885, 0.1891, 0.0579]),
 'persistent_death_hosp': tensor([0.0797, 0.0203, 0.0640, 0.0892, 0.0723]),
 'persistent_I0': tensor([7.1165, 5.4347, 1.9129, 1.0173, 3.8539]),
 'D_state': tensor([[3.5173e-01, 2.7033e+00, 1.5050e+01, 7.9426e+01, 4.1403e+02, 2.1268e+03,
          1.0273e+04, 3.9571e+04, 9.6079e+04],
         [1.4974e-01, 4.9906e-01, 9.9733e-01, 1.6805e+00, 2.6134e+00, 3.8870e+00,
          5.6256e+00, 7.9988e+00, 1.1238e+01],
         [4.8268e-01, 2.1378e+00, 5.9508e+00, 1.4553e+01, 3.3932e+01, 7.7575e+01,
          1.7579e+02, 3.9650e+02, 8.9069e+02],
         [2.1948e-01, 8.9024e-01, 2.0221e+00, 3.8375e+00, 6.7368e+00, 1.1365e+01,
          1.8755e+01, 3.0550e+01, 4.9376e+01],
         [1.1545e-01, 4.2444e-01, 9.1750e-01,

In [5]:
### ENSEMBLE SETUP ###
model_2_path = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/main/data/models/SEIRHD_NPI_Type2_petrinet.json"

In [6]:
result_2 = pyciemss.sample(model_2_path, end_time, logging_step_size, num_samples, start_time=start_time)
result_2['data'].head()

,timepoint_id,sample_id,persistent_beta_c_param,persistent_beta_nc_param,persistent_kappa_param,persistent_gamma_param,persistent_hosp_param,persistent_death_hosp_param,persistent_I0_param,D_state_state,E_state_state,H_state_state,I_state_state,R_state_state,S_state_state,infected_observable_state,exposed_observable_state,hospitalized_observable_state,dead_observable_state
0,0,0,0.777164,0.280381,0.712967,0.165812,0.067952,0.068566,3.470171,0.148899,182.210876,3.139023,119.149956,74.865257,19339664.0,119.149956,182.210876,3.139023,0.148899
1,1,0,0.777164,0.280381,0.712967,0.165812,0.067952,0.068566,3.470171,1.718176,1585.308594,28.053324,1037.077759,751.842834,19336630.0,1037.077759,1585.308594,28.053324,1.718176
2,2,0,0.777164,0.280381,0.712967,0.165812,0.067952,0.068566,3.470171,15.415092,13775.308594,244.098328,9016.912109,6641.261230,19310354.0,9016.912109,13775.308594,244.098328,15.415092
3,3,0,0.777164,0.280381,0.712967,0.165812,0.067952,0.068566,3.470171,134.235809,118195.703125,2111.843994,77767.648438,57643.554688,19084188.0,77767.648438,118195.703125,2111.843994,134.235809
4,4,0,0.777164,0.280381,0.712967,0.165812,0.067952,0.068566,3.470171,1139.344849,912600.312500,17470.417969,626840.625000,483028.375000,17298964.0,626840.625000,912600.312500,17470.417969,1139.344849


In [7]:
### ENSEMBLE SAMPLE INTERFACE ###

model_paths = [model_1_path, model_2_path]
solution_mappings = [lambda x : x, lambda x : x] # Conveniently, these two models operate on exactly the same state space, with the same names.

ensemble_result = pyciemss.ensemble_sample(model_paths, solution_mappings, end_time, logging_step_size, num_samples, start_time=start_time)
ensemble_result['data'].head()

,timepoint_id,sample_id,model_0/persistent_beta_c_param,model_0/persistent_kappa_param,model_0/persistent_gamma_param,model_0/persistent_hosp_param,model_0/persistent_death_hosp_param,model_0/persistent_I0_param,model_1/persistent_beta_c_param,model_1/persistent_beta_nc_param,...,model_0/H_state_state,model_0/I_state_state,model_0/R_state_state,model_0/S_state_state,model_1/D_state_state,model_1/E_state_state,model_1/H_state_state,model_1/I_state_state,model_1/R_state_state,model_1/S_state_state
0,0,0,0.228905,0.697764,0.292566,0.098878,0.078968,14.676838,0.746269,0.611267,...,6.609246,82.220551,117.203323,19339694.0,0.022061,30.251297,1.026544,25.661184,50.608181,19339932.0
1,1,0,0.228905,0.697764,0.292566,0.098878,0.078968,14.676838,0.746269,0.611267,...,29.761646,357.304077,639.404541,19338388.0,0.076176,40.174564,1.590585,34.254513,128.773224,19339832.0
2,2,0,0.228905,0.697764,0.292566,0.098878,0.078968,14.676838,0.746269,0.611267,...,129.447113,1552.223877,2909.138916,19332712.0,0.152206,53.485672,2.148299,45.605007,233.071625,19339706.0
3,3,0,0.228905,0.697764,0.292566,0.098878,0.078968,14.676838,0.746269,0.611267,...,561.963379,6735.284180,12763.881836,19308072.0,0.253972,71.206985,2.864267,60.715481,371.954346,19339534.0
4,4,0,0.228905,0.697764,0.292566,0.098878,0.078968,14.676838,0.746269,0.611267,...,2430.759766,29075.619141,55418.121094,19201888.0,0.389529,94.798378,3.813826,80.831390,556.855896,19339306.0


### Optimize interface
Get infections below 5 individuals at 100 days

In [3]:
from pyciemss.ouu.qoi import scenario2dec_nday_average
from pyciemss.interfaces import optimize

import numpy as np
import torch

risk_bound = 15.
end_time = 100.
qoi = lambda x: scenario2dec_nday_average(x, ["infected_observable"], 1)
objfun = lambda x: np.abs(x)
static_parameter_interventions = {torch.tensor(1.): "beta_c"}
initial_guess_interventions = 0.5
bounds_interventions = [[0.1],[0.8]]
# static_parameter_interventions = {torch.tensor(1.): "gamma"}
# initial_guess_interventions = 0.3
# bounds_interventions = [[0.1],[0.5]]

opt_result = optimize(model_1_path, end_time, logging_step_size, qoi, risk_bound, static_parameter_interventions, objfun,
    initial_guess_interventions=initial_guess_interventions, bounds_interventions=bounds_interventions,
    start_time=0.0, n_samples_ouu=int(5), maxiter=1, maxfeval=10)
opt_result

[0.5] tensor(1.) {'beta_c': tensor(0.5000, dtype=torch.float64)}
[[3.7793517]] [3.7793517]


  0%|          | 0/20 [00:00<?, ?it/s]

[0.5] tensor(1.) {'beta_c': tensor(0.5000, dtype=torch.float64)}
[[3.7793517e+00]
 [3.0736148e-01]
 [9.3792038e+00]
 [4.6508941e+05]
 [4.1046116e+05]] [3.7793517e+00 3.0736148e-01 9.3792038e+00 4.6508941e+05 4.1046116e+05]
[0.5] [3.7793517e+00 3.0736148e-01 9.3792038e+00 4.6508941e+05 4.1046116e+05] 465089.40625
[0.5] tensor(1.) {'beta_c': tensor(0.5000, dtype=torch.float64)}


  5%|▌         | 1/20 [00:52<16:32, 52.22s/it]

[[3.7793517e+00]
 [3.0736148e-01]
 [9.3792038e+00]
 [4.6508941e+05]
 [4.1046116e+05]] [3.7793517e+00 3.0736148e-01 9.3792038e+00 4.6508941e+05 4.1046116e+05]
[0.5] [3.7793517e+00 3.0736148e-01 9.3792038e+00 4.6508941e+05 4.1046116e+05] 465089.40625
[0.57] tensor(1.) {'beta_c': tensor(0.5700, dtype=torch.float64)}


...

 50%|█████     | 10/20 [04:47<04:30, 27.09s/it]

[[3.7343595e+00]
 [3.0213729e-01]
 [9.1142540e+00]
 [4.6469769e+05]
 [3.9484344e+05]] [3.7343595e+00 3.0213729e-01 9.1142540e+00 4.6469769e+05 3.9484344e+05]
[-0.06] [3.7343595e+00 3.0213729e-01 9.1142540e+00 4.6469769e+05 3.9484344e+05] 464697.6875
[0.1] tensor(1.) {'beta_c': tensor(0.1000, dtype=torch.float64)}
[[3.7471900e+00]
 [3.0362582e-01]
 [9.1895761e+00]
 [4.6480978e+05]
 [3.9928588e+05]] [3.7471900e+00 3.0362582e-01 9.1895761e+00 4.6480978e+05 3.9928588e+05]
[0.1] [3.7471900e+00 3.0362582e-01 9.1895761e+00 4.6480978e+05 3.9928588e+05] 464809.78125
[0.1] tensor(1.) {'beta_c': tensor(0.1000, dtype=torch.float64)}


 60%|██████    | 12/20 [05:39<03:32, 26.60s/it]

[[3.7471900e+00]
 [3.0362582e-01]
 [9.1895761e+00]
 [4.6480978e+05]
 [3.9928588e+05]] [3.7471900e+00 3.0362582e-01 9.1895761e+00 4.6480978e+05 3.9928588e+05]
[0.1] [3.7471900e+00 3.0362582e-01 9.1895761e+00 4.6480978e+05 3.9928588e+05] 464809.78125
[0.17] tensor(1.) {'beta_c': tensor(0.1700, dtype=torch.float64)}


KeyboardInterrupt: 